# Test the model, scaler, and encoder

In [8]:
# Imports
from keras.models import load_model
from joblib import load
import pandas as pd
import numpy as np
import warnings

# Ignore the warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the model, scaler, and encoder
model = load_model('model.h5')
scaler = load('min_max_scaler.joblib')
encoder = load('encoder.joblib')

In [40]:
# Single inputs
objid = 1237650000000000000.0 # not there
ra = 183.531326
dec = 0.089693
u = 19.47406
g = 17.0424
r = 15.94699
i = 15.50342
z = 15.22531
run = 752
rerun = 301 # not there
camcol = 4
field = 267
specobjid = 3722360000000000000.0
class_type = 'STAR' # not there for inputs
redshift = -0.000009
plate = 3306
mjd = 54922
fiberid = 491

In [43]:
# Inputs for the model
inputs = [[ra, dec, u, g, r, i, z,
           run, camcol, field, specobjid,
           redshift, plate, mjd, fiberid]]

In [48]:
# Check the example csv file
df = pd.read_csv('example_test_set.csv')
df.sample(3)

,Unnamed: 0,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
6,9339,1.237650e+18,208.101267,0.088784,19.13539,17.86134,17.44055,17.21586,17.14748,752,301,4,431,3.377910e+17,GALAXY,0.023516,300,51943,76
23,6719,1.237650e+18,200.183821,0.337158,19.09781,18.02957,17.49159,17.14819,16.97778,1334,301,4,24,3.344900e+17,GALAXY,0.110840,297,51959,355
54,7622,1.237650e+18,197.961361,67.865715,19.36847,17.42144,16.46410,16.04772,15.69934,1350,301,6,402,7.841040e+18,GALAXY,0.107643,6964,56748,982


In [49]:
# Create inputs from the csv file and respective outputs
inputs = df.drop(['Unnamed: 0', 'rerun', 'class', 'objid'], axis=1)
outputs = df['class']

In [50]:
# Converting the outputs shape to compare it with prediction_class
outputs = outputs.values.reshape(-1 ,1)

In [51]:
# Apply scaler on inputs and get output of softmax probabilities and finally conver it to class
scaled_inputs = scaler.transform(inputs)
prediction = model.predict(scaled_inputs)
prediction_class = encoder.inverse_transform(prediction)

4/4 [==============================] - 0s 3ms/step


In [34]:
# Check where the outputs don't meet the prediction class
np.where(outputs!=prediction_class)[0]

array([ 6, 85, 95])

In [39]:
# Get the classification report
from sklearn.metrics import classification_report
print(classification_report(outputs, prediction_class))

              precision    recall  f1-score   support

      GALAXY       1.00      0.94      0.97        53
         QSO       1.00      1.00      1.00         9
        STAR       0.93      1.00      0.96        38

    accuracy                           0.97       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.97      0.97      0.97       100

